# 🎲 Winning Probabilities – Symbolic Computation

This program computes the winning probabilities **symbolically** for
$p_0,\ldots,p_{m-1}$:
$$ P_A(V,W),\ P_B(V,W),\ P_U(V,W). $$
In addition, for specified field probabilities, the winning probabilities are also reported **exactly** and **numerically**.

---

### ✅ Requirements

- Exactly **one die** is used, with symbolic probabilities ($p_0,\ldots,p_{m-1}$) and user-defined probabilities that sum to $1$.

---

### ⚙️ Inputs

- `p_vals`: user-defined probabilities for the individual fields  
- `S`: betting strategy  
- `T`: betting strategy  

📌 **Caution:** Expressions can grow very quickly (“monster terms”). Use only small chip counts.

---

### 🧩 Module `chip_strategies.py`

The file `chip_strategies.py` contains **all core functions** used for the computations:

- winning probabilities (`P_A`, `P_B`, `P_U`)
- strategy comparisons
- simulations and confidence intervals
- visualizations

> 📌 **Important:** This file must be located in the **root directory**  
> so that it can be imported in the notebooks  
> (e.g. `import chip_strategies as ps`).

The file `chip_strategies.py` can be regarded as a **function library** shared by all Jupyter notebooks in this project.

---

### ▶️ Next Steps

The corresponding Python code is located in the **next code cell**.  
Only the following inputs need to be adjusted:

- `p_vals`: user-defined probabilities for the fields  
  (e.g. `p_vals = (1/2, 1/3, 1/6)`)
- `S`: betting strategy  
  (e.g. `S = (2, 0, 0)`)
- `T`: betting strategy  
  (e.g. `T = (0, 0, 1)`)

> 📌 Make sure that `p_vals`, `S`, and `T` have the same length.

---

### ▶️ Running the Program

- Click inside the code cell.
- Press `Shift + Enter`, to start the computation.  
  Alternatively, click **▶ Run** in the toolbar at the top.
- Use **Run → Run All Cells** to execute all cells at once.

📎 Good luck experimenting with your own strategies!


In [1]:
from functools import lru_cache
from fractions import Fraction
import sympy as sp

# ============================================================
# Input
# ============================================================

p_vals = [Fraction(1,2), Fraction(1,3), Fraction(1,6)]
S = (2, 0, 0)
T = (0, 0, 1)

# ============================================================
# Computation
# ============================================================

def make_symbolic_PA(m):
    """
    Creates P_A(S,T) as a recursive symbolic function in p0...p{m-1}.
    Returns: (P_A_sym, p_symbols)
    """
    # Define p0, p1, ..., p{m-1}
    p_syms = sp.symbols(f"p0:{m}", positive=True)

    @lru_cache(maxsize=None)
    def P_A_sym(S, T):
        S, T = list(S), list(T)
        # Base cases
        if sum(S) == 0 and sum(T) == 0:
            return sp.Integer(0)
        if sum(S) == 0:
            return sp.Integer(1)
        if sum(T) == 0:
            return sp.Integer(0)

        # Normalize only over fields with chips
        s = sum(p_syms[i] for i in range(m) if S[i] or T[i])
        expr = sp.Integer(0)
        for i in range(m):
            if not (S[i] or T[i]):
                continue
            S2 = S.copy(); T2 = T.copy()
            if S2[i] > 0: S2[i] -= 1
            if T2[i] > 0: T2[i] -= 1
            expr += (p_syms[i] / s) * P_A_sym(tuple(S2), tuple(T2))
        return sp.simplify(expr)

    return P_A_sym, p_syms

def compute_symbolic_outcomes(S, T, m):
    """
    Returns the symbolic expressions (P_A, P_B, P_U) in p0...p{m-1}.
    """
    P_A_sym, p_syms = make_symbolic_PA(m)
    PA_expr = P_A_sym(tuple(S), tuple(T))
    PB_expr = P_A_sym(tuple(T), tuple(S))
    PU_expr = sp.simplify(1 - PA_expr - PB_expr)
    return (PA_expr, PB_expr, PU_expr), p_syms

def compute_numeric_outcomes(S, T, p_vals):
    """
    Evaluates the symbolically generated expressions numerically:
    - as sympy.Rational
    - as Python Fraction
    - as float
    Returns a dict containing three tuples.
    """
    m = len(p_vals)

    # 1) Symbolic
    (PA_expr, PB_expr, PU_expr), p_syms = compute_symbolic_outcomes(S, T, m)

    # 2) Substitution in sympy.Rational
    subs = {p_syms[i]: sp.Rational(p_vals[i]) for i in range(m)}
    PA_rat = sp.simplify(PA_expr.subs(subs))
    PB_rat = sp.simplify(PB_expr.subs(subs))
    PU_rat = sp.simplify(PU_expr.subs(subs))

    # 3) Convert to Python Fraction
    PA_frac = Fraction(PA_rat.p, PA_rat.q)
    PB_frac = Fraction(PB_rat.p, PB_rat.q)
    PU_frac = Fraction(PU_rat.p, PU_rat.q)

    # 4) Floating-point values
    PA_f = float(PA_rat)
    PB_f = float(PB_rat)
    PU_f = float(PU_rat)

    return {
        'symbolic': (PA_expr, PB_expr, PU_expr),
        'rational': (PA_frac, PB_frac, PU_frac),
        'float':    (PA_f, PB_f, PU_f)
    }

out = compute_numeric_outcomes(S, T, p_vals)

# ============================================================
# Output
# ============================================================

for mode in ('symbolic', 'rational', 'float'):
    print(f"\n=== {mode.upper()} ===")
    results = out[mode]
    labels  = (f'P_A({S},{T})', f'P_B({S},{T})', f'P_U({S},{T})')
    for lbl, val in zip(labels, results):
        if mode == 'symbolic':
            print(f"\n{lbl} =\n", end=" ")
            sp.pprint(val, use_unicode=True)
        else:
            print(f"{lbl} = {val}")



=== SYMBOLIC ===

P_A((2, 0, 0),(0, 0, 1)) =
      2    
   p₀     
──────────
         2
(p₀ + p₂) 

P_B((2, 0, 0),(0, 0, 1)) =
 p₂⋅(2⋅p₀ + p₂)
──────────────
           2  
  (p₀ + p₂)   

P_U((2, 0, 0),(0, 0, 1)) =
 0

=== RATIONAL ===
P_A((2, 0, 0),(0, 0, 1)) = 9/16
P_B((2, 0, 0),(0, 0, 1)) = 7/16
P_U((2, 0, 0),(0, 0, 1)) = 0

=== FLOAT ===
P_A((2, 0, 0),(0, 0, 1)) = 0.5625
P_B((2, 0, 0),(0, 0, 1)) = 0.4375
P_U((2, 0, 0),(0, 0, 1)) = 0.0
